In [1]:
!pip install xlsxwriter
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 2.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.5 MB/s 
     |████████████████████████████████| 4.9 MB 16.2 MB/s 
     |████████████████████████████████| 1.3 MB 35.0 MB/s 
     |████████████████████████████████| 163 kB 59.1 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f866a3d198561b976e5b6bf3422513ef95c157b732b6bee74f85b83b9bec3cad
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
from bs4 import BeautifulSoup
import requests
import argparse
import random
import pandas as pd
import time
from tqdm.auto import tqdm

마인드카페 게시글 parsing

In [ ]:
for i in tqdm(range(400)):
  path = 'https://www.mindcafe.co.kr'
  community_url = f'{path}/pc/community?counlb=1&page=1'
  story_id = i
  story_url = f'{path}/story?id={story_id}'
  # maximum page : 39512
  # 전문가 의견은 255 까지 있음

  r = requests.get(community_url)
  s = BeautifulSoup(r.text,"html.parser")
  temp = [[],[]]
  for story in s.find_all("div", class_="story"):
    link_ = story.find("a", class_="title")
    if link_:
      c_story = BeautifulSoup(requests.get(path+link_.attrs['href']).text, "html.parser")
      title = c_story.find("div", class_="main__title").text
      contents = c_story.find("div", class_="main__contents").text
      content = title + \
                "\n" +\
                contents
      
      for comment in c_story.find_all("div", class_="comment"):
        temp[0] +=[content] 
        temp[1] += [comment.text]

      temp[0] += [content]
      temp[1] += [c_story.find("div", class_="counselling").text]

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
pd.DataFrame({"Q":temp[0], "A":temp[1]}).to_excel("/content/drive/MyDrive/data_kevin/mindcafe_community_counselling.xlsx")

aha 플랫폼 육아 관련 질문 parsing 하기

In [23]:
import pandas as pd
import sys
# 육아 관련 질문들 373개 페이지
story_id = 1
temp1, temp2 = [],[]
path = 'https://www.a-ha.io/questions'
questions = f'{path}/categories/70?status=published'
print("aha Question page crawling begin")
while 1:
  try:
    questions_url = f'{questions}&page={story_id}'
    # maximum page : 39512
    # 전문가 의견은 255 까지 있음
    r = requests.get(questions_url)
    s = BeautifulSoup(r.text,"html.parser")
    # print(s.findAll("script")[2])
    link_dict = eval(s.findAll("script")[2].text)
    for links in link_dict["mainEntity"]["itemListElement"]:
      requests.get(links['url'])
      r_item = requests.get(links['url'])
      item_s = BeautifulSoup(r_item.text,"html.parser")
      found_contents = item_s.findAll("div", class_="editor__content")
      try:
        title = item_s.find("h1", class_="a-card__headerTitle").text
      except:
        print(f"Error in [ page {story_id} ] this page will skip")
        continue
      user_question = found_contents[0].text
      for content in found_contents[1:]:
        temp1 += [title + "\n" + user_question]
        temp2 += [content.text]
    print(f"aha question [ page {story_id} ] done (scrapped data : {len(temp1)}, {len(temp2)})")
    story_id += 1
    if story_id > 3000:
      print(user_question)
      raise ValueError("page overed")
  except:
    print("End of Page or Some Error in crawling. File will save")
    to_drive = pd.DataFrame({"Q":temp1, "A":temp2})
    print(to_drive)
    to_drive.to_excel("/content/drive/MyDrive/data_kevin/aha_qa_kids.xlsx", index=False)
    to_drive.to_excel("/content/drive/MyDrive/data_kevin/chatbot/aha_qa_kids.xlsx", index=False)
    print("Successfully saved")
    break

aha Question page crawling begin
aha question [ page 1 ] done (scrapped data : 73, 73)
aha question [ page 2 ] done (scrapped data : 133, 133)
aha question [ page 3 ] done (scrapped data : 199, 199)
aha question [ page 4 ] done (scrapped data : 277, 277)
aha question [ page 5 ] done (scrapped data : 358, 358)
aha question [ page 6 ] done (scrapped data : 450, 450)
aha question [ page 7 ] done (scrapped data : 532, 532)
aha question [ page 8 ] done (scrapped data : 621, 621)
aha question [ page 9 ] done (scrapped data : 703, 703)
aha question [ page 10 ] done (scrapped data : 784, 784)
aha question [ page 11 ] done (scrapped data : 862, 862)
aha question [ page 12 ] done (scrapped data : 938, 938)
aha question [ page 13 ] done (scrapped data : 1008, 1008)
aha question [ page 14 ] done (scrapped data : 1082, 1082)
aha question [ page 15 ] done (scrapped data : 1159, 1159)
aha question [ page 16 ] done (scrapped data : 1233, 1233)
aha question [ page 17 ] done (scrapped data : 1304, 1304)

In [ ]:
import pandas as pd
# 심리상담 관련 1958개 페이지
story_id = 1
temp1, temp2 = [],[]
print("aha Question page crawling begin")
while 1:
  try:
    path = 'https://www.a-ha.io/questions'
    questions = f'{path}/categories/57?status=published'
    questions_url = f'{questions}&page={story_id}'
    # maximum page : 39512
    # 전문가 의견은 255 까지 있음
    r = requests.get(questions_url)
    s = BeautifulSoup(r.text,"html.parser")
    link_dict = eval(s.findAll("script")[2].text)    
    for links in link_dict["mainEntity"]["itemListElement"]:
      requests.get(links['url'])
      r_item = requests.get(links['url'])
      item_s = BeautifulSoup(r_item.text,"html.parser")
      found_contents = item_s.findAll("div", class_="editor__content")
      try:
        title = item_s.find("h1", class_="a-card__headerTitle").text
      except:
        print(f"Error in [ page {story_id} ] this page will skip")
        continue
      user_question = found_contents[0].text
      for content in found_contents[1:]:
        temp1 += [title + "\n" + user_question]
        temp2 += [content.text]
    print(f"aha question [ page {story_id} ] done (scrapped data : {len(temp1)}, {len(temp2)})")
    story_id += 1
    if story_id > 5000:
      print(user_question)
      raise ValueError("page overed")    
  except:
    print("End of Page or Some Error in crawling. File will save")
    pd.DataFrame({"Q":temp1, "A":temp2}).to_excel("/content/drive/MyDrive/data_kevin/aha_qa_mind.xlsx", index=False)
    pd.DataFrame({"Q":temp1, "A":temp2}).to_excel("/content/drive/MyDrive/data_kevin/chatbot/aha_qa_mind.xlsx", index=False)
    print("Successfully saved")

    break

aha Question page crawling begin
aha question [ page 1 ] done (scrapped data : 93, 93)
aha question [ page 2 ] done (scrapped data : 179, 179)
aha question [ page 3 ] done (scrapped data : 264, 264)
aha question [ page 4 ] done (scrapped data : 343, 343)
aha question [ page 5 ] done (scrapped data : 394, 394)
aha question [ page 6 ] done (scrapped data : 469, 469)
aha question [ page 7 ] done (scrapped data : 537, 537)
aha question [ page 8 ] done (scrapped data : 606, 606)
aha question [ page 9 ] done (scrapped data : 675, 675)
aha question [ page 10 ] done (scrapped data : 753, 753)
aha question [ page 11 ] done (scrapped data : 814, 814)
aha question [ page 12 ] done (scrapped data : 877, 877)
aha question [ page 13 ] done (scrapped data : 950, 950)
aha question [ page 14 ] done (scrapped data : 1016, 1016)
aha question [ page 15 ] done (scrapped data : 1079, 1079)
aha question [ page 16 ] done (scrapped data : 1169, 1169)
aha question [ page 17 ] done (scrapped data : 1248, 1248)
a

In [ ]:
import pandas as pd

qa = pd.read_excel("/content/drive/MyDrive/data_kevin/aha_qa_kids.xlsx")

In [ ]:
for index, row in qa.iterrows():
  row["Q"] = row["Q"].replace("\n      ","").replace("\n    ","")
  row["A"] = (". ".join(row["A"].split(".")[2:])+".")

In [ ]:
qa.to_excel("/content/drive/MyDrive/data_kevin/chatbot/aha_qa_kids.xlsx", index=False)

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch 

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

QADF = pd.read_excel("/content/drive/MyDrive/data_kevin/chatbot/aha_qa_kids.xlsx")

searcher = SentenceTransformer("Gunulhona/tbsentmodel_v1",#"snunlp/KR-SBERT-V40K-klueNLI-augSTS",
                               device=device)
searcher.tokenizer.model_input_names = ["input_ids", "attention_mask"]

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'Negative', '1': 'Positive'}. The number of labels wil be overwritten to 2.


In [ ]:
embed_filename = "qa_embeddings2"

In [ ]:
import pickle

temp = searcher.encode(QADF["Q"].unique(), convert_to_tensor=True)
with open(embed_filename, "wb") as f:
  pickle.dump(temp, f)
  f.close()


In [ ]:
import pickle

with open("qa_embeddings", "wb") as f:
  pickle.dump(temp, f)
  f.close()

In [ ]:
import pickle

embed_filename ="/content/drive/MyDrive/data_kevin/qa_embeddings"
with open(embed_filename, "rb") as f:
  embed = pickle.load(f)

In [ ]:
que = [
  "출산 직후 아기의 건강에 문제가 있었습니까?",
  "3세 이전에 자녀는 어떤 기질이였는지 적어주세요.",
  "현재까지 자녀의 건강문제가 있었습니까?",
  "현재 자녀에게 어떤 증상이나 어려움이 있습니까?",
  "현재 자녀가 집착하는 것이 있습니까?",
  "자녀가 무서워하는 것이 있습니까?",
  "지금까지 가족의 커다란 환경 변화가 있었다면 구체적으로 무엇인가요?",
  "자녀가 가정 내에서 가장 어려워하는 사람은 누구이며, 그 이유는 무엇입니까?",
  "자녀는 형제들과 잘 지냈습니까?",
  "자녀와 애정표현을 하고 함께 시간을 보내며 대화를 많이 하는 편인가요?",
  "자녀가 유치원이나 학교에 갈 때 등원 혹은 등교 거부가 있었습니까?",
  "숙제나 준비물 챙기기, 주변 정리, 씻기 등 대부분의 일을 잘 처리할 수 있습니까?",
  "학습능력에 있어 자녀가 잘 알아 듣고 가르쳐 주면 잘 따라 했나요? 학교 진도를 잘 따라갈 수 있습니까?",
  "다른 사람의 말을 잘 이해하고 또래 수준에 맞는 의사소통이 가능한가요?",
  "자녀가 문제 상황이나 갈등 상황에서 어떻게 대처하나요?",
  "자녀는 학교에서 수업시간에 집중하고, 규칙을 잘 지키는 편인가요?",
  "학부모 상담시 자녀에 대한 학교 선생님의 평가는 어떠했습니까?",
  "자녀가 학교에 잘 적응하는 편인가요?",
  "자녀는 평상 시 자신의 욕구나 감정을 잘 표현하는 편입니까?",
  "자녀가 스트레스를 받으면 어떻게 하나요",
  "상황에 맞지 않게 감정을 과도하게 표현하는 경우가 있나요?",
  "지금까지 자녀에게 중요한 사건이 있었다면 구체적으로 무엇인가요?",
  "자녀가 또래들로부터 따돌림이나 괴롭힘을 당한 적이 있습니까?",
  "자녀는 평소에 다른 아이들과 잘 지내는 편인가요?",
  "자녀는 대체로 어떤 성향입니까?",
  "자녀의 장점은 무엇입니까?",
  "자녀의 단점은 무엇입니까?",
  "자녀에 대해 하고 싶은 말이 있으면 자유롭게 말씀해 주세요."
  ]


In [ ]:
from sentence_transformers import util
temp = []
results = util.semantic_search(searcher.encode(que), embed)
for result in results:
  searched = QADF.Q.unique()[result[0]["corpus_id"]]
  answers = QADF.loc[QADF.Q == searched, "A"]
  print(searched)
  temp += [answers.sample(3).tolist()]

남아 5세인데 태권도 다닐수 있나요?
태권도는거의 필수로 가야한다고 들었는데요가면 공놀이나 체육활동도 하는거 같던데격한 운동을 할수있을까 살짝 걱정되네요보통 몇살부터 잘 가나요?
18개월 집에서 까치발로 걷는데 문제없나요
18개월쯤에 접어들면서부터 까치발로 걷고있어요 걸음마 연습할때 한번씩 그랬었고 잘걷기 시작하면서는 괜찮았는데 지금 18개월인데 집에서는 무조건 까치발로 걷네요 재미로 그런것 같긴한데 하지말라고해도 안듣고 계속 걸어요 무슨문제는 없는건지 안하게 할수는 없는지 궁금해요
태권도띠 순서가 어떻게 되나요?
첫째가 태권도를 시작한지 한달정도 되었는데다른 형들이랑 누나가 다른띠여서 그런지 태권도띠 순서를 물어보는데 제가 잘몰라서 문의를 드립니다.
10개월 아기 빈혈수치 12 철분수치 22래요
안녕하세요10개월 1주차 아기입니다남자아기고요월요일 피검사 했고요 (철분, 염증, B형간염 항체검사)검사 당일엔 아기 빈혈 없다고 3~4일 후정확한 피검사 결과를 들으러 오라고 했어요오늘 검사 결과를 듣는데제가 의학적 이야기는 문외한이라알아들은 거 두가지는일단 빈혈수치는 12.몇으로 정상수치이다단 무엇무엇이 30이 정상인데 22라서철분 1cc 씩 석달을 먹여야 한다 라고 하시더라구요빈혈 수치가 정상이라면 지금 철분을 잘 사용하고 있단 뜻인지아니면 철분이 모자라서 문제가 생겼다는 뜻인지 ㅠㅠ철분이 부족하면 머리가 나빠질 수도 있다고 해서조심한다고 했는데...이런 결과가 나와서 너무 슬픕니다 ㅠㅠ..
초등학생아이가 운동을 하는데 그만둔다고 하면 어떻게 얘기하는게 좋을까요?
초등학교 아들이 운동을합니다.. 좋아서 시작한건데 가끔 너무 힘들어서 그만둔다고 하는데 이럴때는존중하고 그만두게 하는게 맞는지 아니면 포기하지않도록 하는게 맞는지 .. 어떻게 얘기하는게 좋은 방법일까요??
전국에 자폐증 관련 특수학교가 어디잇나요?
저희 큰애가 자폐증이라 구미혜당학교 초등부에 다니고 있습니다. 구미엔 특수학교가 한곳뿐이라 여러장애인들이 같이 수업을 받고있습니다. 혹시 자폐만 전문으로 하

In [ ]:
temp

[['대략 5세부터 아이를 받는곳이 많습니다. 너무 걱정된다면 우선 태권도 관장님과 이야기를 나누는것이 좋을수있으며체험을 시켜보는것도 좋겠습니다. .',
  '그렇게 격하디 않습니다 5세 정도면 무리없이 운동을 따라올수 있기때문에다닐 수 있습니다.',
  '5살 아이들도 많이 다닙니다.  품새 배우고 예절 배우고 즐겁게 놀이 체육도 하고 격한 운동은 많이 없어요 필수라서 가기보다는 아이가 좋아한다면 보내도 될것같습니다. 보통 초등 저학년때 많이 보내더라구요.  .'],
 ['생후 10개월에서 18개우러 사이의 아이가 까치발로 걷는 것은 대부분 걸음마를 배울 때 일시적으로 나타나는 자연스러운 현상입니다.  아직 아이가 18개월 정도밖에 안되는만큼 너무 염려하실 필요는 없을 것 같습니다. 그러나 간혹 뇌 문제가 원인일 수도 있는데 대표적으로 뇌성마비인 경우 까치발을 걷는 경우가 있습니다.  뇌성마비를 앓으면 신체 균형과 움직임을 조절하는 뇌에 문제가 생겨 까치발을 걷기도 하며,.  선천적으로 아킬레스건에 이상이 있거나 양쪽 다리 길이가 다른 경우에도 까치발을 걸을 수 있습니다. 그러나 아이의 다리가 뻣뻣한 느낌이 아니라면 크게 걱정하지 않아도 되니 너무 염려하시지 말길 바랍니다. 참고서적=《출동! 우리아기 홈닥터》 (세브란스 어린이병원 지음).',
  '발바닥에 뭔가 묻는게 싫어서 까치발을 할 수도 있구요 어머님 말씀처럼 재미있어서 하는 경우도 있습니다또는 발레 영상을 보고 따라 하는 경우도 있구요 까치발을 했을 때 느낌이 뭔가 다르니 재미로 할 수도 있습니다혹시 아이가 혼자 쪼그려 앉았을 때 발 뒤꿈치가 바닥에 안닿으면  신체 어딘가에 문제 있는 거라 병원 가봐야 하는 걸로 알고 있어요그런게 아니면 자연스러운 현상이니 너무 걱정 안하셔도 될 것 같아요 .',
  '까치발로 걷는 아이 걱정이 되셔서 질문 남겨주셨네요. 혹시 걱정하시는 이유가 혹시 우리 아이가 자폐아 일까? 라는 걱정이실까요?부모님께서 불안하시다면 가정에서 체크해보실 수 있는 것들이 있습니다. 1.

In [ ]:
from gensim.summarization.summarizer import summarize
text = list()
for i, x in enumerate(temp):
  temp[i] = "".join(x).replace(". .",".")
  summ = summarize(temp[i], ratio=0.3)
  text += [summ]
  print(summ)

품새 배우고 예절 배우고 즐겁게 놀이 체육도 하고 격한 운동은 많이 없어요 필수라서 가기보다는 아이가 좋아한다면 보내도 될것같습니다.
그러나 간혹 뇌 문제가 원인일 수도 있는데 대표적으로 뇌성마비인 경우 까치발을 걷는 경우가 있습니다.
선천적으로 아킬레스건에 이상이 있거나 양쪽 다리 길이가 다른 경우에도 까치발을 걸을 수 있습니다.
우리아기 홈닥터》 (세브란스 어린이병원 지음).발바닥에 뭔가 묻는게 싫어서 까치발을 할 수도 있구요 어머님 말씀처럼 재미있어서 하는 경우도 있습니다또는 발레 영상을 보고 따라 하는 경우도 있구요 까치발을 했을 때 느낌이 뭔가 다르니 재미로 할 수도 있습니다혹시 아이가 혼자 쪼그려 앉았을 때 발 뒤꿈치가 바닥에 안닿으면  신체 어딘가에 문제 있는 거라 병원 가봐야 하는 걸로 알고 있어요그런게 아니면 자연스러운 현상이니 너무 걱정 안하셔도 될 것 같아요 .까치발로 걷는 아이 걱정이 되셔서 질문 남겨주셨네요.
까치발은 있는 경우도 있고 없는 경우도 있습니다.
그리고 단순히 재미있어서 하는 경우도 있고요, 걸음마시기에 까치발을 들었을 때 중심이 잘 잡혀진 경험이 있다면 그 습관을 계속해서 고수하는 경우도 있습니다.
발바닥 감촉을 둔하게 만드는 것도 방법일 수 있습니다.

지금부터라도 병원에서 말한 부족한 성분들을 아이에게섭취하게 한다면 문제가 없을 것입니다.
보통 혈액검사에서 <Fe>가 철분 수치이고, 빈혈은 말씀하신 혈색소로 확인가능한데왜 검사결과가 늦게 나온<Ferritin>을 중요하게 보냐면이 수치가 <저장철>을 말하기 때문입니다.
절대 지능이나 건강에 문제가 있을 수치가 전혀 아니며,단지 정상수치 중에서도 조금 낮은 편에 포함되는 것으로담당선생님께서 미리 예방차원에서 단기간 철분을 복용하자고 친절하게 알려주신듯합니다.
아이가 자신의 진로를 결정하느니 만큼 진지하게 다시 생각해볼 수있도록 하는 것이 좋을 것 같습니다운동을 그만두고 싶은 이유그만두면 나중에 하고싶은일하고싶은 일이 있다면 그만두지않고 열심히 할 수있는지공부를 할 거라면 

연예인을 하겠다는데, 구체적인 계획이 있는지 확인해보시고, 없다면 최소한 구체적인 계획을 세워볼 것을 권해보시기 바랍니다.
일단 아이와 많은 대화를 통하여 이야기 해보는 것이좋을 것 같으니 참고하시길 바랍니다.아이에게 어느 정도의 열정이 있는지 확인하는 것이 좋을 것 같습니다니가 진짜 하고 싶다면 그만한 재능과 열정을 나에게 보여줘야 내가 널 지원해주지 않겠니?이렇게 말이죠 그냥 발만 담그고 싶어서 그런거라면 지원해주실 이유가 없습니다 단호하게 말씀하셔야합니다.

의외로 체중이 정상인 아이에서도 빈혈이 있는 경우가 매우 많습니다.
4살 이전에는 하루 12시간 이상의 수면을 요하며초등학교 입학 전후에도 10-12시간 정도 수면을 취해야 성장에 도움이 됩니다.
키성장이 해당 연령(월령이 더 정확합니다)에서 10퍼센타일 이하이면서최근 6개월간 키 성장이 3cm가 되지 않는다면반드시 골연령 검사와 성장호르몬 검사를 받아야합니다.
그러나 많은 분들이 이러한 성장호르몬 검사를 받아야할 중요한 시기임에도많은 건강보조식품이나 보조 영양제가키를 키워준다든지, 무조건 면역을 증강시켜준다든지하는 허위광고 등을 의지해 중요한 치료 시기를 놓칩니다.
꼭 기본적인 성장검사와 매년 빈혈검사  + 비티만 D + 충분한 수면이 원칙만 지켜주시면 아이는 충분히 자랄 수 있을 것입니다.
도움이 되셨길 바랍니다.

쪽쪽이에 대해서는 다양하고 상반된 이론들이 많습니다.

유난히 승부욕 강한 기질도 있고, 그렇지만 타고난 성격이라고 인정하는 거랑 그러니 그냥 둬도 된다라는 거랑은 다르기 때문에 바로 잡아줘야 합니다.
이런 성향의 아이들은 강압적으로 아이를 꺾기보단 아이와 같은 편에 서서 아이의 속상한 마음을 같이 달래주고 스스로 다른 방법을 생각해 볼 수 있도록 도와주셔야 합니다.
그래야지 주변 사람들의 관계에 있어서도 이겨야 할 적이 아니라 서로 도와가는 관계라고 느낄 수 있어야 합니다.
.승패에 집착하는 아이들의 경우 무엇보다 융통성을 키워줘야 합니다.
그리고 되도록 경쟁 상황에는 노출시키지 마

In [ ]:
searched=\
{
    "신체발달": " ".join(text[0:3]),   # 신체발달
    "증상파악": " ".join(text[3:6]),   # 증상파악
    "가족문제": " ".join(text[6:10]),  # 가족문제
    "학교생활": " ".join(text[10:18]), # 학교생활
    "정서발달": " ".join(text[18:22]), # 정서발달
    "사회성": " ".join(text[22:25]),   # 사회성
    "기타": " ".join(text[25:])        # 기타
 } 

In [ ]:
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
for key_name in searched:
  raw_input_ids = tokenizer.encode(searched[key_name])
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  input_ids = torch.tensor([input_ids])
  summary_ids = model.generate(input_ids,
                              num_beams=4,
                              max_length=512)
  print(key_name, ": ", tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True))
  print(summarize(searched[key_name], ratio=0.5))
  print()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


신체발달 :  간혹 뇌성마비인 경우 까치발을 걷는 경우가 많을 수도 있는데 대표적으로 뇌성마비인 경우 까치발을 걷는 경우가 많을 수도 있고 발바닥에 뭔가 묻는게 싫어서 까치발을 할 수도 있고 발레 영상을 보고 따라 하는 경우도 있고 발레 영상을 보고 따라 하는 경우도 있구요
그러나 간혹 뇌 문제가 원인일 수도 있는데 대표적으로 뇌성마비인 경우 까치발을 걷는 경우가 있습니다.
까치발은 있는 경우도 있고 없는 경우도 있습니다.
그리고 단순히 재미있어서 하는 경우도 있고요, 걸음마시기에 까치발을 들었을 때 중심이 잘 잡혀진 경험이 있다면 그 습관을 계속해서 고수하는 경우도 있습니다.



증상파악 :  아이가 자신의 진로를 결정하느니만큼 진지하게 다시 생각해볼 수있도록 하는 것이 좋을 것 같으며 아이가 자신의 진로를 결정하느니만큼 진지하게 다시 생각해볼 수있도록 하는 것이 좋을 것 같다.
보통 혈액검사에서 <Fe>가 철분 수치이고, 빈혈은 말씀하신 혈색소로 확인가능한데왜 검사결과가 늦게 나온<Ferritin>을 중요하게 보냐면이 수치가 <저장철>을 말하기 때문입니다.
절대 지능이나 건강에 문제가 있을 수치가 전혀 아니며,단지 정상수치 중에서도 조금 낮은 편에 포함되는 것으로담당선생님께서 미리 예방차원에서 단기간 철분을 복용하자고 친절하게 알려주신듯합니다.



가족문제 :  시중에 변기와 관련된 책도 쇄도하고 있지만 변기 훈련은 18개월~23 정도에 시작하는 것이 좋지만, 24개월 전후에 시작하는 경우가 대부분이랍니다.
시중에 변기와 관련된 책도 있습니다.
아이에게 어느 정도의 열정이 있는지 확인하는 것이 좋을 것 같습니다니가 진짜 하고 싶다면 그만한 재능과 열정을 나에게 보여줘야 내가 널 지원해주지 않겠니?이렇게 말이죠 그냥 발만 담그고 싶어서 그런거라면 지원해주실 이유가 없습니다 단호하게 말씀하셔야합니다.자녀와 대화를 해보시기 바랍니다.
연예인을 하겠다는데, 구체적인 계획이 있는지 확인해보시고, 없다면 최소한 구체적인 계획을 세워볼 것을 권해보시기 바랍니다.

학교생활 :  아이의 행동교정을 위해 지속적으로 사과를 하지 않는 행동을 할 때에 적절한 훈육을 해주시기 바란다.
아이의 행동교정을 위해 지속적으로 사과를 하지 않는 행동을 할 때에 적절한 훈육을 해주시기 바랍니다.
아이도 부모님의 인정을 받기 위하여 한번이라도 더 먹으려고 할 것이고칭찬과 이에 대한 포상을 적절히 해주시면 아이도 성취감도 느끼고 식사를잘 할 것입니다.일단 아이에게 식사의 즐거움을 알게 해주어야 합니다.
아이가 잘 먹었던 것을 종이에 적어놓으시기 바랍니다.
한번이라도 먹었던 음식은 적어놓습니다.
일단 아이와 많은 대화를 통하여 이야기 해보는 것이좋을 것 같으니 참고하시길 바랍니다.아이에게 어느 정도의 열정이 있는지 확인하는 것이 좋을 것 같습니다니가 진짜 하고 싶다면 그만한 재능과 열정을 나에게 보여줘야 내가 널 지원해주지 않겠니?이렇게 말이죠 그냥 발만 담그고 싶어서 그런거라면 지원해주실 이유가 없습니다 단호하게 말씀하셔야합니다.
키성장이 해당 연령(월령이 더 정확합니다)에서 10퍼센타일 이하이면서최근 6개월간 키 성장이 3cm가 되지 않는다면반드시 골연령 검사와 성장호르몬 검사를 받아야합니다.
그러나 많은 분들이 이러한 성장호르몬 검사를 받아야할 중요한 시기임에도많은 건강보조식품이나 보조 영양제가키를 키워준다든지, 무조건 면역을 증강시켜

정서발달 :  쪽쪽이에 대해서는 다양하고 상반된 이론들이 많지만 유난히 승부욕 강한 기질도 있고, 타고난 성격이라고 인정하는 거랑 그러니 그냥 둬도 된다라는 거랑은 다르기 때문에 바로 잡아줘야 한다.
그래야지 주변 사람들의 관계에 있어서도 이겨야 할 적이 아니라 서로 도와가는 관계라고 느낄 수 있어야 합니다.
.승패에 집착하는 아이들의 경우 무엇보다 융통성을 키워줘야 합니다.
그리고 되도록 경쟁 상황에는 노출시키지 마세요.



사회성 :  돌토는 『아기가 태어나면』에서 여자아이는 남자아이보다 말을 훨씬 빨리 배운다고 주장하였으며, 아이를 훈육할 때에는 간단하고 확고하게 아이가 이해하기 쉽도록 전달해야하며 구구절절 설명하기 보다는 짧고 명료한 단어나 짧은 문장으로 아이에게 잘못된 행동을 강조하는 말이 아닌긍정행동을 말해주어야 한다고 말했다.
다만 아이를 훈육할 때에는 간단하고 확고하게 아이가 이해하기 쉽도록 전달해야하며 구구절절 설명하기 보다는 짧고 명료한 단어나 짧은 문장으로 아이에게 잘못된 행동을 강조하는 말이 아닌긍정행동을 말해주어야 하기 대문에 대체로 훈육시 반말을 사용하고는 합니다.
아주 굶기는 건 또 아니지만요.일단은 치즈나 바나나를 아이에게 주는 것을 점차 줄이시고아이가 좋아하는 반찬을 찾아보시는 것이 좋을 것 같습니다.
아이가 좋아하는 것들을 위주로 다양하게 요리하는 방법으로식사량을 조금씩 늘리는 방향으로 해야할 것 같습니다.



기타 :  키와 관련해서는 영양가 있는 음식을 섭취해야 하며, 잠을 푹 자야 키가 커지며 잠을 푹 자야 키가 커지기 때문에 부정교합의 검진 시기는 앞니 영구치가 처음으로 나는 만 6~8세 전후가 좋으며 부정교합의 검진 시기는 부정교합의 검진 시기는 부정교합의 검진 시기는 부정교합의 검진 시기가 처음으로 나는 만 6~8세 전후가 좋다.
.아이가 운동을 그만두려고하는 진짜 이유가 무엇인지 확인해 보시기 바랍니다.



In [ ]:
for i, x in enumerate(temp):
  raw_input_ids = tokenizer.encode(x)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  input_ids = torch.tensor([input_ids])

  summary_ids = model.generate(input_ids,
                              num_beams=4,
                              max_length=512)
  print(tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True))


In [ ]:
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('ainize/kobart-news')
model = BartForConditionalGeneration.from_pretrained('ainize/kobart-news')
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
input_ids = torch.tensor([input_ids])

summary_ids = model.generate(input_ids,
                             num_beams=4,
                             max_length=512)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
for i, x in enumerate(temp):
  raw_input_ids = tokenizer.encode(x)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  input_ids = torch.tensor([input_ids])

  summary_ids = model.generate(input_ids,
                              num_beams=4,
                              max_length=512)
  print(tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True))

아기는 잠드는 것이 어려우며 잠들었다 하더라도 자주 깨는 경우가 많고  아이는 상대적으로 얕은 잠을 자기 때문에 악몽을 꾸거나 바깥의 자극에 예민하게 반응하여 잠이 든 후 1~2시간 사이에 울거나 뒤척이는 일이 잦기 때문에 깬 다음 스스로 잠이 들지 못하고 누가 옆에 있거나 도와주어야 한다는 것이다.
아연과 비타민D가 면역기능, 세포성장 등에 꼭 필요한 성분인 아연은 하루 한번만 먹이면 되고 작아서 애들이 거부감이 없어 면역기능과 세포성장 등에 꼭 필요한 성분이며 아이들이 면역기능과 골격 형성이되어가는 시기로 아연이 결핍되면 성장의 지연이 생길 수 있어서 반드시 챙겨주어야 한다.
남가고용평등법에서 보장하는 육아휴직은 현재 재직하는 직장에서 180일이상 근무한 근로자에 대해서는 근로자의 신청이 있는 경우, 사업주는 이를 거부할 수 없고, 재직기간 180일 미만자는 회사가 육아휴직의 부여를 거부할 권리가 있지만 현 직장에서의 육아휴직 사용시 피보험단위기간이 180일 이기때문에 육아휴직및 적용을 받기 위해선 180일 경과를 거쳐야 한다.
아이가 어리다면 여러가지 감각을 깨울수있는 애플비 사에 촉감그림책을 추천드립니다.
이번 정부에서 만5세 입학으로 입학 나이를 조정하는것을 발표했는데 만나이를 도입하는것을 공약사항으로 내세워었는데 그것과 관련이 있을 것 같고 정책의 추후에도 바뀔 수 있기에 일단은 행정부에서 어떤 움직임을 보일지 지켜봐야겠습니다.
해외리콥탈수면의 장점은 아기와 엄마아빠 수면질을 상승시키고아기의 자립심을 기를수 있지만 단점으로는 처음부터 분리수면을 하면 아이가 불안감에 잠을 자지 못하는 등 정서적으로 불안해 할수 있는데 함께 수면을 시도하면서 아이가 좋아하는 이불이나 인형 등을 배치하여 아이의 관심이 있게하는 것으로 시작하시는 것이 좋다.
엄상을 갖추고 타인에 대한 배려와 예절 교육을 아이를 위해 해보시고 그 이후에 한글이나 예체능 등을 교육해주세요.
최등학생이라면 절약의 습관부터 길러주세요. 소비와 지출의 통제만 배워도 아주 좋은경제교육인  얼마만큼의